# NCBI Datasets - Retrieving ant orthologs with v14

### Table of contents REVIEW
* [Part I: Accessing genomes](#Part-I)
* [Part II: Accessing genes](#Part-II)
* [Part III: Accessing orthologs](#Part-III)
* [Part IV: Building a BLAST database and creating a phylogenetic tree](#Part-IV)
* [Part V: Downloading large datasets (dehydration/rehydration)](#Part-V)

### Important resources
- Github: https://github.com/ncbi/datasets/training
- NCBI datasets: https://www.ncbi.nlm.nih.gov/datasets/

## Before we start... What is a jupyter notebook?

Jupyter Notebooks are a web-based approach to interactive code. A single notebook (the file you are currently reading) is composed of many "cells" which can contain either text, or code. To navigate between cells, either click, or use the arrow keys on your keyboard.

A text cell will look like... well... this! While a code cell will look something like what you see below. To run the code inside a code cell, click on it, then click the "Run" button at the top of the screen. Try it on the code cell below!

In [ ]:
#This is a code cell
print('You ran the code cell!')

If it worked, you should have seen text pop up underneath the cell saying `You ran the code cell!`. Note the `In [1]:` that appeared next to the cell. This tells you the order you have run code cells throughout the notebook. The next time you run a code cell, it will say `In [2]:`, then `In [3]:` and so on... This will help you know if/when code has been run.

The remainder of the notebook below has been pre-built by the workshop organizer. You will not need to create any new cells, and you will be explicitly told if/when to execute a code cell.

The code in this workshop is either Bash (i.e., terminal commands) or Python. Bash commands are prefixed with `!` or the cells have the notation `%%bash` at the top., while Python commands are not. If you are not familiar with code, don't feel pressured to interpret it very deeply. Descriptions of each code block will be provided!

(Jupyter Notebook explanation by Cooper Park at the workshop on [Finding and Analyzing Metagenomic Data](https://www.nlm.nih.gov/oet/ed/ncbi/2021_10_meta.html))

## Case study: Elmo loves ants

Elmo is a graduate student at the Via Sesamum University. As part of his Ph.D. project, he studies Panamanian leaf cutter ants (genus *Acromyrmex*, family Formicidae) and how variation in the gene *orco* (**o**dorant **r**eceptor **co**receptor) affects the colonies of this genus.

(here's the [link](https://www.ncbi.nlm.nih.gov/labs/pmc/articles/PMC5556950/) to a cool paper talking about this gene in ants of the species *Ooceraea biroi*).

![ants](https://github.com/ncbi/datasets/blob/master/training/cshl-2021/images/ants.png?raw=true)

Elmo will use `datasets` to help him gather the existing genomic resources from NCBI. He will:

- download all available genomes for the genus *Acromyrmex*
- download the *orco* gene from the *Acromyrmex* reference genome
- download the ortholog set for this gene for all ants (Formicidae)

In addition, he will also do the following tasks:
- Create a custom BLAST database with the Panamanian leaf cutter ants genomes 
- BLAST the gene *orco* against the database
- Multiple sequence alignment of the BLAST results and the ortholog gene sequences
- Build a phylogenetic tree

### How is `datasets` organized?

[NCBI datasets](https://www.ncbi.nlm.nih.gov/datasets/docs/v1/quickstarts/command-line-tools/) is a command line tool that allows users to download data packages (data + metadata) or look at metadata summaries for genomes, RefSeq annotated genes, curated ortholog sets and virus genome sequences and SARS-Cov-2 proteins. The program follows a hierarchy that makes it easier for users to select exact which options they would like to use. In addition to the program commands, additional flags are available for filtering the results. We will go over those during this tutorial.
<img src="./images/datasets1.png" alt="datasets" style="width: 700px;"/>

### `dataformat`

Now we are going to combine `datasets` with another tool called `dataformat`. `dataformat` allows you to extract metadata information from the JSON-Lines data report files included with all `datasets` data packages or accessible through `summary` command. You can use `dataformat` to:
- Create a tab-delimited file (.tsv) or excel file with the fields you need
- Quickly visualize the information on the screen

<img src="./images/dataformat1.png" alt="dataformat"  style="width: 700px;"/>

In [ ]:
%%bash
# Read the dataformat help menu. This is a great way to get a list of the available metadata fields.
dataformat tsv genome -h

## Part I: Accessing genomes<a class="anchor" id="Part-I"></a>

<img src="./images/elmo_workflow.png" style="width: 600px;">

First, let's figure out what kind of genome information NCBI has for ants (family Formicidae). For this task, we will use the command `datasets summary` command, as shown in the diagram below. We will pipe the `datasets` output to `jq` so we can see how many ant genomes are in the NCBI database.

<img src="./images/datasets-s-genome-tax.png" style="width: 800px;"/>

In [ ]:
%%bash
# Get metadata info
datasets summary genome taxon formicidae | jq '.total_count'


Now we know that there are 108 ant genomes at NCBI. Let's save the full metadata file as JSON-Lines using the flag `--as-json-lines` to make it easier to extract information from the metadata file later.

In [ ]:
%%bash
# Get metadata info and save to a file
datasets summary genome taxon formicidae --as-json-lines > formicidae_summary.jsonl

**Now let's take a look at the metadata using jq**  
Since the output is really long, we will only show the first 50 lines (`head -n 50`). The flag `-C` in the `jq` command shows the output in color, which maes it easier to read.

In [ ]:
%%bash

cat formicidae_summary.jsonl | jq -C . | head -n 50


### A little bit more about JSON files

A JSON (JavaScript Object Notation) file stores data structures and objects. In a very simplified (and non-technical) way, a JSON file is a box, that might contain other boxes with more boxes inside. The same is true for JSON-Lines; the difference is that in a JSON-Lines file, each line represents a full JSON object.  

At the first level of `datasets summary genome` our JSON-Lines "box" is organized like this:
 
```
{
 assemblies[
      assembly{},
      assembly{},
 ],
 total_count
}
```
If we want to look at the value in the field "total_count", here's the command we would use:

In [ ]:
%%bash
datasets summary genome taxon herpestidae | jq '.total_count'

As we mentioned above, in addition to `datasets`, we have a companion tool called `dataformat`. `dataformat` converts `datasets` JSON-Lines outputs to `tsv` or Excel <html>&reg;</html> formats. Similarly to our JSON reports, `dataformat` also follows a hierarchical organization.
<br>
<img src="./images/dataformat-genome.png" style="width:900px;"/>



For example: if we want to explore some  basic assembly stats (such as contig N50), we can will find that information in the box `assmstats` (`assm` = assembly). This image is useful to help you build the field names `dataformat` recognizes. In this example, the field name has two parts: `assmstats` and `contig-n50`, and those will be joined by a dash (`-`), like this: `assmstats-contig-n50`.
<br>
<img src="./images/dataformat-genome-contign50.png" style="width:900px;"/>


In [ ]:
%%bash
datasets summary genome taxon herpestidae --as-json-lines | dataformat tsv genome --fields assmstats-contig-n50

#The equivalent jq command is shown below:
#datasets summary genome taxon herpestidae --as-json-lines | jq '.assembly_stats.contig_n50'

Now let's see how we can retrieve the scientific names associated with those assemblies.
<br>
<img src="./images/dataformat-genome-org.png" style="width:900px;"/>

In [ ]:
%%bash
datasets summary genome taxon herpestidae --as-json-lines | dataformat tsv genome --fields organism-name

#jq equivalent:
#datasets summary genome taxon herpestidae --as-json-lines | jq '.organism.organism_name'

As you can see, `dataformat` is very useful in retrieving information from the summary metadata. Let's try a few more complex examples.

First, let's retrieve information from three fields at the same time: scientific name, biosample accession number  and contig N50.
<br>
<img src="./images/dataformat-genome-3fields.png" style="width:900px;"/>

In [ ]:
%%bash

datasets summary genome taxon herpestidae --as-json-lines | dataformat tsv genome \
--fields organism-name,assminfo-biosample-accession,assmstats-contig-n50


**Last one**: let's look at a larger collection of genome assemblies (let's say, all Carnivora) and select only those assemblies with contig N50 larger than 15 Mb (15000000 bp). `datasets` provides many options for filtering, but there is no built-in filter for contig N50 size.  

Here's what we want to see: assembly accession number, species and assembly level for those genomes with contig N50 above 15 Mb. Notice that assembly accession is different from biosample accession.

<br>
<img src="./images/dataformat-genome-3fields-accession.png" style="width:900px;"/>

In [ ]:
%%bash

datasets summary genome taxon carnivora --as-json-lines | dataformat tsv genome \
--fields accession,organism-name,assminfo-level,assmstats-contig-n50 > carnivora.tsv

#datasets summary genome taxon carnivora | jq -r '.reports[] 
#| select(.assembly_stats.contig_n50 > 15000000) 
#| [.accession, .organism.organism_name, .assembly_info.assembly_level, .assembly_stats.contig_n50] 
#| @tsv'

In [ ]:
import pandas as pd                                   #load pandas to this notebook

carnivora = pd.read_csv('carnivora.tsv', sep='\t')    #use pandas to import carnivora.tsv
carnivora                                             #visualize the data table as the object carnivora

Using `pandas`, we will select only the rows with contig N50 value above 15000000. 

In [ ]:
#Filter the column Assembly Stats Contig N50 by showing only accessions with contig N50 larger thab 15 Mb.

carnivora[carnivora['Assembly Stats Contig N50'] > 15000000]

### Let's continue to explore the available genomes for the family Formicidae


For this part, we will use two UNIX commands: `sort` and `uniq`. 

- `sort` can be used to sort text files line by line, numerically and alphabetically.   
- `uniq` will filter out the repeated lines in a file. However, `uniq` can only detect repeated lines if they are adjacent to each other. In other words, if they are alphabetically or numerically sorted. The flag `-c` or `--count` tells the command `uniq` to remove the repeated lines, and to count how many times each value appeared. 

So, we will use `dataformat` to extract the information we need, sort the result and count the number of unique entries.

In [ ]:
%%bash
# For which species does NCBI have genomes in its database? How many per species?

datasets summary genome taxon formicidae --as-json-lines | dataformat tsv genome \
--fields organism-name --elide-header | sort | uniq -c

In [ ]:
%%bash
# What is the assembly level (contig, scaffold, chromosome, complete) breakdown?

datasets summary genome taxon formicidae --as-json-lines | dataformat tsv genome \
--fields assminfo-level --elide-header  | sort | uniq -c

### How to get help when using the command line

Since `datasets` is a very hierarchical program, we can use that characteristic to our advantage to get very specific help.   For example: if we type `datasets --help`, we will see the first level of commands available.


In [ ]:
%%bash
datasets --help

Notice the difference from when we type `datasets summary genome taxon formicidae --help`  


In [ ]:
%%bash
datasets summary genome taxon formicidae --help

### Exercises

Now we will practice what we learned about `datasets` and `dataformat`. Take a look at the questions below and feel free to ask questions. Check out the `--help` options from the command line to assist you in this exercise.


In [ ]:
%%bash
# How many reference genomes in the family Formicidae? (hint --reference)



In [ ]:
%%bash
# How many reference genomes are annotated? (hint: --annotated)



In [ ]:
%%bash
# How many genomes have NCBI (RefSeq) annotations? (hint: --assembly-source)



### Bonus questions:

In [ ]:
%%bash
# Now look at the summary metadata for your organism of interest 
# (if you don't have a favorite, go with red panda, Ailurus fulgens, taxid: 9649)



In [ ]:
%%bash
# How many genomes?



In [ ]:
%%bash
# Assembly level breakdown



In [ ]:
%%bash
# How many genomes have a contig N50 value above 15Mb?



### What is the difference/relationship between Genbank, RefSeq and Reference assemblies?
  
   
<br>

---
<img src="https://www.ncbi.nlm.nih.gov/datasets/docs/v2/images/gcf_vs_gca_v9.png" alt="difference-gca-gcf" style="width:600px;">

---

### Data package

We explored the `datasets summary` option, in which we had a chance to look at the summary metadata ***without*** downloading any files. In the next steps, we will look at the data packages, which contain the actual data files. 

<br>

<img src="./images/data-packages.png" alt="data_package" style="width:600px;" />
<br>

Each `datasets` option (aka. `genome`, `gene`, `virus`) has a different data package content. Users can customize data packages by using the flag `--include` to add or remove files. The image below shows the filetypes (potentially) available for each `datasets` option. 
  
The same flag (`--include`) is used to add multiple data reports to the data package. Please refer to the [NCBI Datasets Data Package Reference page](https://www.ncbi.nlm.nih.gov/datasets/docs/v2/reference-docs/data-packages/) for more information about each report and data package contents.

<br>

<img src="./images/data-package-contents-op2.png" style="width:800px;">


---
#### IMPORTANT:

When the flag `--include` is invoked, the default package contents are reset, which means that we need to list all file types we want to download.  

For example: the default genome data package contains the JSON-Lines data report `assembly_data_report.jsonl` and each genome assembly sequences in FASTA format. Let's imagine we also want to download the protein sequences. In that case, we would use the following command:

`datasets download genome taxon cat --include protein,genome`

If we say: 
`datasets download genome taxon cat --include protein`

Only the protein sequences would be included, but not the genome assembly FASTA. 

---

In [ ]:
%%bash
# Download all available GenBank assemblies for the genus Acromyrmex and save as genomes.zip

datasets download genome taxon acromyrmex --assembly-source genbank --filename genomes.zip --no-progressbar

In [ ]:
%%bash
# Unzip genomes.zip to the folder genomes
unzip -o genomes.zip -d genomes

In [ ]:
%%bash
# Explore the folder structure of the folder genome with the command tree
tree -C genomes/

### Let's recap our goals

We used `datasets` to download all the Genbank assemblies for the genus *Acromyrmex*. The next step is to download the gene *orco* (odorance receptor coreceptor) for the same genus. But first, let's learn more about how genes are organized at NCBI.

<img src="./images/elmo_workflow-1.png" alt="done1" style="width: 600px;" />

## Part II: Accessing genes <a class="anchor" id="Part-II"></a>
### GENES

Independent of choosing `datasets download` or `datasets summary`, there are four options for retrieving gene information: `accession`, `gene-id`, `symbol` and `taxon`. 


<img src="./images/datasets-gene.png" style="width: 700px;"/>


When choosing any of those options, you will retrieve the gene information for the **reference** taxon. Like this:

`datasets download gene accession XR_002738142.1`  
`datasets download gene gene-id 101081937`  
`datasets download gene symbol BRCA1 --taxon cat`  
`datasets download gene taxon cat`. 

The first three commands will download the same gene from the cat (<i>Felis catus</i>) <u>reference genome</u>, and the last one will download all genes that species RefSeq annotation. 

- **accession**:  Unique identifier. Accession includes RefSeq RNA and protein accessions. Since it's unique, taxon is implied (aka there will never be two sequences from different taxa with the same accession number).  

- **gene-id**:  Also a unique identifier. Every RefSeq genome annotated has a unique set of identifiers. For example: the gene-id for BRCA1 in human is 672, while in cat is 101081937.  

- **symbol**: Differently from accession and gene-id, gene symbol is not unique and means different things in different taxonomic groups. If using the symbol option, you should specify the species. The default option is human.

- **taxon**: Species-level. Retrieves the entire set of RefSeq annotated genes for the specified taxon.  

**Remember**  
Both `summary` and `download` will return results for the **reference assembly** of a <u>single species</u>. If you want to download a curated set of the same gene for multiple taxa, you should use the flag `--ortholog`. We'll talk more about it later. 

<img src="./images/dataformat-gene.png" />

Now let's take a look at a gene example:

In [ ]:
%%bash
#Example: IFNG in human
datasets summary gene symbol ifng | jq -C .


In [ ]:
%%bash
# how datasets deals with synonyms
datasets summary gene symbol IFG --as-json-lines | dataformat tsv gene \
--fields tax-name,symbol,synonyms


In [ ]:
%%bash
#Example: IFNG in cat
datasets summary gene symbol ifng --taxon "felis catus"


### Back to ants
We will download the gene *orco* for the species *Acromyrmex echinatior*. We will use the gene-id 105147775 instead of the symbol because no informative gene symbol has been assigned for this gene.  

In [ ]:
%%bash
# Using gene-id to retrieve gene information
datasets summary gene gene-id 105147775 --as-json-lines | dataformat tsv gene \
--fields description,gene-id,symbol,tax-name


In [ ]:
%%bash
# if we try to retrieve metadata information for this gene using the symbol orco, what happens?
datasets summary gene symbol orco --taxon "acromyrmex echinatior"


In [ ]:
%%bash
# Download the gene data package for the gene-id 105147775 (*orco* in Acromyrmex echinatior)
# We want to include the FASTA file with gene sequences, so we will use the flag --include.

datasets download gene gene-id 105147775 --filename gene.zip --include gene,protein,rna --no-progressbar


In [ ]:
%%bash
#Unzip the file
unzip -o gene.zip -d gene

In [ ]:
%%bash
#Explore the data package structure using tree
tree gene

### Exercises

1. Look for the summary data for a gene of interest (check the [etherpad](https://etherpad.wikimedia.org/p/CSHL_Datasets_Workshop_2021) for suggestions)
2. What is the gene location?
3. What is the gene range?
4. Now, download a list of gene symbols using the file genes.txt (provided). Save it as gene_list.zip
5. Unzip gene_list.zip and explore the folder structure
6. How many fasta files are there?

In [ ]:
%%bash
# Summary data



In [ ]:
%%bash
# Gene location



In [ ]:
%%bash
# Gene range



In [ ]:
%%bash
# Download a list of genes and save the data package as gene_list.zip (--filename gene_list.zip)


In [ ]:
%%bash
# Explore the folder structure



In [ ]:
%%bash
# How many genes were downloaded?



In [ ]:
%%bash
# How many fasta files in the data package?



## Part III: Accessing orthologs <a class="anchor" id="Part-III"></a>

### Orthologs

Since `datasets version 14`, users can retrieve ortholog information using the flag `--ortholog` under the `gene` service (with exception for the `taxon` option).

#### <font color='blue'>Wait, but what is an ortholog set?</font>

>An ortholog set, or ortholog gene group, is a group of sequences that have been identified by the NCBI genome annotation team as homologous genes related to each other by speciation events. They are identified by a combination of protein similarity + local syntheny information. 
Currently, NCBI has ortholog sets calculated for vertebrates and some insects. 


#### Examples:

`datasets download gene accession XR_002738142.1 --ortholog all`  
`datasets download gene gene-id 101081937 --ortholog all`  
`datasets download gene symbol BRCA1 --ortholog all --taxon cat`  

All three commands will download the **same** ortholog set (which is the complete set). 

What if I want to filter the ortholog set to include *only* a taxonomic group of interest?



### Applying a taxonomic filter to the ortholog set

When using the `--ortholog` flag, users need to provide an argument for it. The argument should be one or more taxa (any rank) to filter results or 'all' for the complete set.

#### Examples

- `datasets summary gene symbol brca1 --ortholog "felis catus"`  
Prints a json metadata summary of the gene brca1 for the domestic cat only. 
  
  
- `datasets summary gene symbol brca1 --taxon "felis catus" --ortholog all`  
Even though this option looks similar to the one above, the result is *very different*. Here, we're asking `datasets` to find the ortholog set to which the gene brca1 in the domestic cat belongs. And `datasets` will download the <u>entire</u> ortholog set, not only the sequences for the domestic cat.


#### We are going to do the following steps:
- download the ortholog data package and save it with the name ortholog.zip
- unzip it to the folder ortholog
- look at the files

Helpful info:

- gene symbol: orco
- gene-id in *Drosophila melanogaster*: 40650
- gene-id in *Acromyrmer echinatior*: 105147775
- target taxon: Formicidae

In [ ]:
%%bash
# download the orco ortholog set for ants (Formicidae)
datasets download gene gene-id 40650 --ortholog formicidae --include gene --filename ortholog.zip --no-progressbar


In [ ]:
%%bash
# unzip it to the folder ortholog
unzip -o ortholog.zip -d ortholog


In [ ]:
%%bash
#Explore the folder structure
tree ortholog/


## What have we done so far?
- Explored metadata for all ant genomes
- Downloaded genomes for the panamanian leaf cutter ant
- Downloaded the *orco* gene for *Acromyrmex echinatior*
- Downloaded the ortholog set for all ants for the *orco* gene

<br>

<img src="./images/elmo_workflow-2.png" style="width:600px;"/>

## Part IV: Building a BLAST database and creating a phylogenetic tree<a class="anchor" id="Part-IV"></a>

### Here's what we are showing you now:
- BLAST:
    - Create a BLAST database for each genome
    - BLAST the *orco* gene sequence against the genomes database and extract the matching regions
- multiple sequence alignment of the blast matches and the ortholog sequences
- generate a approximate maximum likelihood tree using FastTree

We'll add more detailed information about the commands we're using here to the GitHub page.

#### Extracting taxIDs from the genome data package

First, let's use `dataformat` to extract the species names, taxID and assembly accession numbers from the genomes we downloaded. We will talk in more detail about `dataformat` later.

In [ ]:
%%bash
# Extract tax id for each species:
dataformat tsv genome --fields organism-name,organism-tax-id,accession --package genomes.zip 

#### Creating a BLAST database with taxonomy information.

First we are going to create a folder called `blastdb` with the UNIX command `mkdir`. Next, we will change to the directory we just created. Finally, we will make a copy of the NCBI taxonomy database (taxdb)

In [ ]:
# Create a folder called blastdb
!mkdir blastdb

# change directory to the folder blastdb
%cd blastdb

# download the NCBI Taxonomy Database (taxdb)
!update_blastdb.pl taxdb

#Now let's extract the taxdb to the blastdb folder
!tar xfz taxdb.tar.gz

#### BLAST database and search
Now we will create a BLAST database with the *Acromyrmex* genomes we downloaded. More information about the commands is available on out GitHub page.

In [ ]:
!pwd

In [ ]:
%%bash
# Create a blast database for each genome
makeblastdb -dbtype nucl -in ../genomes/ncbi_dataset/data/GCA_000204515.1/GCA_000204515.1_Aech_3.9_genomic.fna -taxid 103372 -out Aechinatior
makeblastdb -dbtype nucl -in ../genomes/ncbi_dataset/data/GCA_017607455.1/GCA_017607455.1_ASM1760745v1_genomic.fna -taxid 230686 -out Ainsinuator 
makeblastdb -dbtype nucl -in ../genomes/ncbi_dataset/data/GCA_017607545.1/GCA_017607545.1_ASM1760754v1_genomic.fna -taxid 2715315 -out Acharruanus
makeblastdb -dbtype nucl -in ../genomes/ncbi_dataset/data/GCA_017607565.1/GCA_017607565.1_ASM1760756v1_genomic.fna -taxid 230685 -out Aheyeri
makeblastdb -dbtype nucl -in ../genomes/ncbi_dataset/data/GCA_024713525.1/GCA_024713525.1_ASM2471352v1_genomic.fna -taxid 103372 -out Aechinatior

# Create an alias under which the four genome databases can be called
blastdb_aliastool -dbtype nucl -title acromyrmex -out acromyrmex -dblist "Acharruanus Aechinatior Aheyeri Ainsinuator"

# BLASTN search
blastn \
-db acromyrmex \
-query ../gene/ncbi_dataset/data/gene.fna \
-evalue 1e-50 \
-outfmt 11 \
-max_hsps 1 \
-out orco_acromyrmex_1e-50.asn

# Covert the asn.1 output to tabular (output format 6)

blast_formatter \
-archive orco_acromyrmex_1e-50.asn \
-outfmt '6 sseqid sstart send evalue length staxid ssciname' > orco_acromyrmex_1e-50.tsv

Using `pandas` again, we will create an object with the tsv file we just created from the BLAST output, so we can take a look at our results.

In [ ]:
# Create a table and visualize the BLAST results

blast_table = pd.read_csv('orco_acromyrmex_1e-50.tsv', sep='\t', header=None)
blast_table

#### Converting from BLAST to fasta

Now we are going to use some "tricks" (not really, just some good old bash scripting) to extract fasta sequences from the BLAST output. For tthis task, we will be using `blast_formatter` again.

In [ ]:
%%bash
# Convert BLAST output to fasta

blast_formatter \
-archive orco_acromyrmex_1e-50.asn \
-outfmt '6 ssciname sseqid sseq' \
-max_target_seqs 4 | awk 'BEGIN{FS="\t"; OFS="\n"}{gsub(/ /, "_", $1);gsub(/-/, "", $3); print ">"$1"_"$2,$3}' > ../acromyrmex_orco.fasta


<img src="./images/elmo_workflow-3.png" style="width: 600px;"/>

### VERY IMPORTANT!
For the next steps, we need to go back to our home folder. Let's do it in steps again.

In [ ]:
%%bash
## Check where you are
pwd

In [ ]:
## If you're not in the home folder, run this command:
%cd ..

### Multiple sequence alignment: BLAST matches + *orco* orthologs

First, let's simplify the FASTA headers in the ortholog set.

In [ ]:
%%bash
# Extract the FASTA headers from the gene ortholog fasta, replace the spaces by commas and save as a txt file.
grep ">" ortholog/ncbi_dataset/data/gene.fna | sed 's/ /,/g' > ortholog_seqid.txt

# Create a mapping file with the original name in the column 1 and a shortened name on column 2
# For the new name, we will keep only the genus initial, species name and NCBI Gene ID. Example:
# Original: NW_017296201.1:c2116125-2107387 LOC108758761 [organism=Trachymyrmex cornetzi] [GeneID=108758761] [chromosome=Un]
# New name: T_cornetzi_108758761

cat ortholog_seqid.txt | while read line; do
new=$( echo $line | awk 'BEGIN {FS=","; OFS="_"}{gsub(/\[organism\=/, "", $3);gsub(/]/, "", $4);gsub(/\[GeneID\=|\]/, "", $5)} ;{print substr($3,1,1)"_"$4,$5}'); 
old=$( echo $line | sed 's/,/\_/g;s/>//g')
printf "${old}\t${new}\n" >> name_map.tsv; 
done

# Copy the ortholog dataset fasta from the folder ortholog to the folder datasets
cp ortholog/ncbi_dataset/data/gene.fna ortholog_gene.fna

# Replace spaces in the FASTA header by underscores.
# Why? Because we need the FASTA headers to match the "old" in the file name_map.tsv. This is what we have:
# Original: >NW_017296201.1:c2116125-2107387,LOC108758761,[organism=Trachymyrmex,cornetzi],[GeneID=108758761],[chromosome=Un]
# "old": NW_017296201.1:c2116125-2107387_LOC108758761_[organism=Trachymyrmex_cornetzi]_[GeneID=108758761]_[chromosome=Un]
# "new": T_cornetzi_108758761

sed 's/ /_/g' ortholog_gene.fna > ortholog_gene_nospaces.fna

#Replace the names in the fasta file
cat ortholog_gene_nospaces.fna | seqkit replace \
--kv-file  <(cut -f 1,2 name_map.tsv) \
--pattern "^(.*)" --replacement "{kv}" > ortholog_gene_final.fna

Let's check the first ten FASTA headers after the name replacement. 

In [ ]:
%%bash

grep ">" ortholog_gene_final.fna | head

### Multiple sequence alignment and phylogenetic reconstruction

Now, let's concatenate the FASTA we extracted from the BLAST matches, align them using MAFFT and use FastTree to generate an approximate ML phylogeny.

In [ ]:
%%bash

#Concatenate sequences
cat ortholog_gene_final.fna acromyrmex_orco.fasta > orco_all.fasta

#align sequences with mafft
mafft orco_all.fasta > orco_all_aln.fasta

#Generate a phylogeny using fasttree
FastTree -nt orco_all_aln.fasta > orco.tree

### Visualizing the tree

In [ ]:
import Bio
from Bio import Phylo
from Bio.Phylo.Applications import PhymlCommandline
from Bio.Phylo.PAML import codeml
from Bio.Phylo.PhyloXML import Phylogeny

import matplotlib as mp
%matplotlib inline

In [ ]:
# Read tree file

orco_xml = Phylo.convert("orco.tree", "newick", "orco.tree.xml", "phyloxml")
orco_tree = Phylo.read("orco.tree.xml", "phyloxml")

orco_tree_rooted = orco_tree.root_with_outgroup('O_brunneus_116854080','D_quadriceps_106748868','H_saltator_105183395')
Phylo.draw_ascii(orco_tree)


## Part V: Downloading large datasets (dehydration/rehydration) <a class="anchor" id="Part-V"></a>

Now you learned how to download genomes, genes and ortholog gene sets from NCBI with one command using `datasets`. Now we want to show you another feature of `datasets` that allows you to download what we call a `dehydrated` package. Let's download a dehydrated package and explore the files inside it.

In [ ]:
%%bash
# Download a dehydrated data package for all acromyrmex GenBank genomes
datasets download genome taxon acromyrmex --assembly-source genbank --dehydrated --include genome,protein --filename acromyrmex.zip --no-progressbar

In [ ]:
%%bash
# Next we have to unzip the dehydrated package
unzip -o acromyrmex.zip -d acromyrmex

In [ ]:
%%bash
# Now let's use the command tree to look at the data package contents
tree acromyrmex/

**What is difference between this folder (`acromyrmex`) and the folder `genomes`?**   
Let's use `tree` again to look at the contents of the folder genomes.

In [ ]:
%%bash
# Check the folder contents of genome
tree genomes/

Both packages include the files `assembly_data_report.jsonl` and `dataset_catalog.json`, but the folder acromyrmex has the file `fetch.txt` instead of the *actual* data. Let's take a look in this file.

In [ ]:
# Inspect the file fetch.txt
import pandas as pd
fetch = pd.read_csv('./acromyrmex/ncbi_dataset/fetch.txt', sep='\t', header=None)
fetch

The file `fetch.txt` has a list of files to be "fetched" (downloaded) with their respective links. And they are the same files that were originally included in when we downloaded the genomes in the beginning of this notebook.  

#### BUT WHY WOULD I WANT TO USE THIS OPTION?

Some possibilities:
- You are working with very large genomes and want to share the data with your collaborators. Instead of sending a massive data file, you can send a text file that they can use to download the same data you're working on.
- Or maybe you hand selected some genomes for a project from the [NCBI Datasets website](https://www.ncbi.nlm.nih.gov/datasets/genomes/) and they don't follow a specific pattern that can be replicated. You can also download a dehydrated package from our website, share it and download everything you need later.

In [ ]:
%%bash
# Let's look at the help file for rehydrate
datasets rehydrate -h

In [ ]:
%%bash
# Let's get a list of files that are available for download 
datasets rehydrate --directory acromyrmex/ --list

In [ ]:
%%bash
# Let's only get the protein sequences for the genome with the highest contigN50 value
datasets rehydrate --directory acromyrmex/ --match GCA_000204515.1/protein.faa 

In [ ]:
%%bash
# Let's use tree to look at our folder acromyrmex again
tree acromyrmex/

We can see that the file we requested ` GCA_000204515.1/protein.faa` was downloaded to the folder `acromyrmex`

In [ ]:
%%bash
# Take a peek at the downloaded protein file
cat acromyrmex/ncbi_dataset/data/GCA_000204515.1/protein.faa | head

## Exercise
* Download a dehydrated package for all *Mycobacterium tuberculosis* genomes that meet all of the following criteria (hint: use flags)
    1. submitted/released in 2021
    2. annotated
    3. assembly level of complete_genome
* use `dataformat` to view the sequencing technology used for each of these genomes
* use the `rehydrate` option to get the genome sequence for one genome generated using Oxford Nanopore

In [ ]:
%%bash
# Download a dehydrated genome data package



In [ ]:
%%bash
# Unzip the data package



In [ ]:
%%bash
# Use dataformat to generate a table that includes sequencing technology



In [ ]:
%%bash
# Use rehydrate to get genome sequence generated using Oxford Nanopore

